In [1]:
%run init.ipynb

Using TensorFlow backend.
/home/fanyixing/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/fanyixing/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3692.72it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2321.82it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1304983.28it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 2957.05it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:10<00:00, 1802.23it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 24798.55it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 2853.24it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [3]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f6841eb1f98>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f68966e4f98>,
 'vocab_size': 16674,
 'embedding_input_dim': 16674,
 'input_shapes': [(10,), (100,)]}

In [4]:
#%ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
#%ranking_task.metrics = [
#%    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
#%    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
#%    mz.metrics.MeanAveragePrecision()
#%]

In [5]:
bin_size = 30
model = mz.models.DRMM()
model.params['input_shapes'] = [[10,], [10, bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________________________________________________________________________
attention_

In [6]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [7]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_generator = mz.HistogramDataGenerator(data_pack=test_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [10]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, bin_size, 'LCH',
                                                num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 12s 122ms/step - loss: 0.8916
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.45799517445184007 - normalized_discounted_cumulative_gain@5(0.0): 0.5303670277792623 - mean_average_precision(0.0): 0.48704074730779007
Epoch 2/30
102/102 [==============================] - 19s 188ms/step - loss: 0.7349
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5178921074119822 - normalized_discounted_cumulative_gain@5(0.0): 0.5871083692532375 - mean_average_precision(0.0): 0.5406443320610673
Epoch 3/30
102/102 [==============================] - 24s 234ms/step - loss: 0.6446
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5683260761340332 - normalized_discounted_cumulative_gain@5(0.0): 0.6300620832774922 - mean_average_precision(0.0): 0.5847452108496706
Epoch 4/30
102/102 [==============================] - 18s 173ms/step - loss: 0.5556
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5916930041572077 - n

In [12]:
drmm_model = mz.load_model('./drmm_pretrained_model/16')
test_generator = mz.HistogramDataGenerator(data_pack=dev_pack_processed[:10],
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
test_x, test_y = test_generator[:]
prediction = drmm_model.predict(test_x)
prediction

array([[ 1.0377446 ],
       [ 0.8148954 ],
       [-0.29903623],
       [ 0.84281605],
       [-0.43238235],
       [-0.06015391],
       [-0.81013334],
       [ 0.06438022],
       [-1.401072  ],
       [ 0.2644889 ]], dtype=float32)

In [13]:
import shutil
shutil.rmtree('./drmm_pretrained_model/')